In [1]:
from agents import Context, MyAgent
import agentformats
import agenttools

In [2]:
agent = MyAgent(name="Agente", context=Context(output_channel="stream", output_format="markdown"),
    role_and_skills="Sei un operatore incaricato di cercare di rispondere in italiano alle domande che ricevi.")
agent.do("Come ti chiami?")

---
**Agente**: Mi chiamo Gemma, sono un modello linguistico di grandi dimensioni creato da Google DeepMind.

In [6]:
context = Context()
agent1 = MyAgent(name="Agente 1", context=context,
    role_and_skills="Sei un operatore incaricato di cercare di rispondere in italiano alle domande che ricevi.")
agent2 = MyAgent(name="Agente 2", context=context,
    role_and_skills="Sei un operatore incaricato di cercare di rispondere in inglese alle domande che ricevi.")
agent3 = MyAgent(name="Agente 3", context=context,
    role_and_skills="Sei un operatore incaricato di cercare di rispondere in francese alle domande che ricevi.")
agent1.do("Conosci il nome di cinque nazioni europee?")
agent2.do("E tu conosci il nome di tre nazioni europee diverse da quelle già indicate?")
agent3.do("E tu conosci il nome di quattro nazioni europee diverse da tutte quelle già indicate?")

---
**Agente 1**: Certo! Ecco il nome di cinque nazioni europee:

1.  Italia
2.  Francia
3.  Spagna
4.  Germania
5.  Regno Unito 

Hai altre domande su di esse o sulla geografia europea in generale?


---
**Agente 2**: Absolutely! Here are three more European countries, different from the ones I mentioned before:

1.  Poland
2.  Romania
3.  Netherlands 

Do you want to test me with another question, or perhaps talk about these countries further?

---
**Agente 3**: Bien sûr ! Voici quatre autres pays européens différents :

1.  Portugal
2.  Slovénie
3.  Croatie
4.  Chypre 

Est-ce que vous voulez me poser une autre question ou en parler davantage ?

In [3]:
agent = MyAgent(name="Agente", context=Context(), response_format=agentformats.list_schema,
    role_and_skills="Sei un operatore incaricato di cercare di rispondere alle domande che ricevi.")
agent.do("Conosci il nome di cinque nazioni europee?")

---
**Agente**: {"elenco": ["Italia", "Francia", "Germania", "Spagna", "Regno Unito"]}


In [3]:
agent = MyAgent(name="Agente", context=Context(), tools=[agenttools.list_files, agenttools.read_file],
    role_and_skills="Sei un operatore incaricato di cercare informazione nel file system locale.")
agent.do("Puoi elencare i files contenuti nella directory /home/lucamari/prova?")
agent.do("Puoi mostrare il contenuto del primo file che hai elencato?")

---
**Agente**: ['uno.txt', 'due.txt']

---
**Agente**: questo è il contenuto del file 'uno.txt'


In [9]:
agent = MyAgent(name="Agente", context=Context(), tools=[agenttools.generate_random_number],
    role_and_skills="Sei un operatore incaricato di generare numeri casuali.")
agent.do("Puoi un numero casuale tra 10 e 12?")

---
**Agente**: 11.738198222758465

In [12]:
context = Context()
agent1 = MyAgent(name="Agente 1", context=context, tools=[agenttools.get_current_time],
    role_and_skills="Sei un operatore incaricato di gestire l'orario.")
agent2 = MyAgent(name="Agente 2", context=context,
    role_and_skills="Sei un operatore incaricato di cercare di rispondere in buon italiano alle domande che ricevi.")
agent1.do("Che ore sono?")
agent2.do("Prendi l'informazione prodotta dall'Agente 1 e la scrivi in una frase in italiano, indicando solo giorno e mese, ore e minuti?")

---
**Agente 1**: 2025-04-20 20:05:40.508334

---
**Agente 2**: Sono le 20:05 del 20 aprile.

In [14]:
context = Context()
agent1 = MyAgent(name="Agente 1", context=context, tools=[agenttools.evaluate_expression],
    role_and_skills="""Sei un operatore incaricato di scrivere e poi calcolare espressioni matematiche.""")
agent2 = MyAgent(name="Agente 2", context=context,
    role_and_skills="Sei un operatore incaricato di cercare di rispondere in buon italiano alle domande che ricevi.")
agent1.do("Quanto vale 1234 diviso 7?")
agent2.do("Prendi il numero prodotto dall'Agente 1 e lo arrotondi a 4 cifre decimali?")

---
**Agente 1**: 176.28571428571428

---
**Agente 2**: 176.2857

In [17]:
context=Context()
agent1 = MyAgent(name="Agente 1", context=context, tools=[agenttools.web_search],
    role_and_skills="Sei un operatore incaricato di fare ricerche nel web.")
agent2 = MyAgent(name="Agente 2", context=context,
    role_and_skills="""Sei un operatore incaricato di rispondere in buon italiano alle domande che ricevi
        formattando la risposta in Markdown, senza alcuna instestazione, commento o spiegazione,
        e organizzando il testo in modo che possa essere facilmente convertito in una presentazione,
        in particolare con un titolo di livello 1 (#) per ogni nuova slide,
        e considerando che ogni slide dovrebbe contenere una quantità limitata di testo.""")
agent3 = MyAgent(name="Agente 3", context=context, tools=[agenttools.markdown_to_pptx],
    role_and_skills="Sei un operatore incaricato di convertire in PowerPoint dei testi in formato Markdown.")
agent1.do("Chi era Santiago Ramon-y-Cajal?")
agent2.do("Scrivi un testo di sintesi nell'informazione raccolta dall'Agente 1 e lo formatti in Markdown come specificato?")
agent3.do("Crei una presentazione in PowerPoint a partire dal testo prodotto dall'Agente 2?")

---
**Agente 1**: [{'title': 'Santiago Ramón y Cajal - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Santiago_Ramón_y_Cajal', 'body': 'Santiago Ramón y Cajal (Spanish: [sanˈtjaɣo raˈmon i kaˈxal]; 1 May 1852 - 17 October 1934) [1] [2] was a Spanish neuroscientist, pathologist, and histologist specializing in neuroanatomy and the central nervous system.He and Camillo Golgi received the Nobel Prize in Physiology or Medicine in 1906. [3] Ramón y Cajal was the first Spaniard to win a scientific Nobel Prize.'}, {'title': 'Santiago Ramón y Cajal | Neuroscience, Neuroanatomy & Histology ...', 'href': 'https://www.britannica.com/biography/Santiago-Ramon-y-Cajal', 'body': "Santiago Ramón y Cajal (born May 1, 1852, Petilla de Aragón, Spain—died Oct. 17, 1934, Madrid) was a Spanish histologist who (with Camillo Golgi) received the 1906 Nobel Prize for Physiology or Medicine for establishing the neuron, or nerve cell, as the basic unit of nervous structure.This finding was instrumental in the recognition of the neuron's fundamental role in nervous function ..."}, {'title': 'The Father of Modern Neuroscience Discovered the Basic Unit of the ...', 'href': 'https://www.scientificamerican.com/article/the-father-of-modern-neuroscience-discovered-the-basic-unit-of-the-nervous-system/', 'body': 'H our after hour, year after year, Santiago Ramón y Cajal sat alone in his home laboratory, head bowed and back hunched, his black eyes staring down the barrel of a microscope, the sole object ...'}]

---
**Agente 2**: # Santiago Ramón y Cajal: Il Pionere della Neuroscienza

Santiago Ramón y Cajal (1852-1934) fu un neuroscienziato, patologo e histologo spagnolo. È considerato il padre della moderna neuroscienza grazie alla sua rivoluzionaria scoperta che il neurone è l'unità fondamentale del sistema nervoso centrale. 

Prima di Ramón y Cajal, Camillo Golgi proponeva una visione "diffusa" del sistema nervoso, dove le cellule nervose erano interconnesse in un'unica massa. Attraverso anni di meticolosa osservazione microscopica e tecniche histologiche innovative, Ramón y Cajal dimostrò l'esistenza di neuroni individuali, separati da sinapsi, che comunicavano tra loro.

La sua scoperta ha permesso di comprendere il funzionamento del sistema nervoso in modo completamente nuovo e ha aperto la strada a ulteriori ricerche nel campo della neuroscienza. Per il suo lavoro, Ramón y Cajal condivise il Premio Nobel per la fisiologia o la medicina nel 1906 con Camillo Golgi.


---
**Agente 3**: santiago_cajal.pptx

In [18]:
context=Context()
agent1 = MyAgent(name="Creativo", context=context,
    role_and_skills="Sei il creativo dell'organizzazione, incaricato di produrre idee interessanti per le richieste che ricevi, dandone anche una giustificazione.")
agent2 = MyAgent(name="Analista", context=context,
    role_and_skills="Sei l'analista dell'organizzazione, incaricato di studiare, commentare criticamente, e se occorre suggerire decisioni.")
agent3 = MyAgent(name="Decisore", context=context,
    role_and_skills="Sei il decisore dell'organizzazione, incaricato di decidere sulle proposte che ricevi.")
agent1.do("Proponi quattro titoli per un seminario sui possibili cambiamenti antropologici indotti dalla diffusione della GenAI.")
agent2.do("Scegli i due titoli che ti sembrano migliori e giustifica la tua scelta.")
agent3.do("Decidi il titolo che ti sembra migliore e giustifica la tua decisione.")

---
**Creativo**: Assolutamente! Ecco quattro proposte di titoli per un seminario sui cambiamenti antropologici indotti dalla GenAI, con la relativa giustificazione del perché ciascuno potrebbe essere efficace:

**1. Titolo:** "L'Uomo Rivisto: Antropologia e l'Era della Generative Intelligence"

*   **Giustificazione:** Questo titolo è audace e provocatorio. “L’Uomo Rivisto” suggerisce un cambiamento fondamentale nella nostra comprensione di cosa significa essere umani, in relazione a una tecnologia che sta ridefinendo la creatività, il lavoro e l'interazione sociale. L'aggiunta di "Antropologia" chiarisce immediatamente lo scopo del seminario, mentre "Era della Generative Intelligence" è un termine tecnico riconoscibile che crea un senso di urgenza e rilevanza per il pubblico.

**2. Titolo:** "Oltre l’Algoritmo: Antropologia, Intelligenza Artificiale Generativa e il Futuro dell'Identità"

*   **Giustificazione:** Questo titolo affronta direttamente una delle preoccupazioni centrali: come l'IA generativa sta rimodellando la nostra identità personale e collettiva. "Oltre l’Algoritmo" implica che ci sarà qualcosa di più profondo da esplorare rispetto alla semplice analisi tecnica dell'IA, invitando a un approccio antropologico. La frase finale – “Futuro dell’Identità” – è potente e attrae l'attenzione su una delle conseguenze più rilevanti per le società umane.

**3. Titolo:** "Dall’Arte alla Realtà: Antropologia e le Nuove Forme di Cultura Generativa"

*   **Giustificazione:** Questo titolo si concentra sull'aspetto culturale dell'IA generativa, evidenziando come questa tecnologia stia trasformando il modo in cui creiamo arte, musica, letteratura e altre forme di espressione culturale. "Dall’Arte alla Realtà” suggerisce un percorso che va dalle manifestazioni più estetiche a un impatto più ampio sulla nostra esperienza della realtà. “Cultura Generativa” è un termine chiave che cattura l'essenza del cambiamento in atto.

**4. Titolo:** “La Macchina come Mediatore: Antropologia e le Implicazioni Sociali dell'Intelligenza Artificiale Generativa”

*   **Giustificazione:** Questo titolo adotta una prospettiva più analitica, concentrandosi sul ruolo dell’IA generativa come mediatrice tra gli esseri umani. “La Macchina come Mediatore” suggerisce che l'IA non è solo uno strumento, ma un agente attivo nella nostra vita sociale e culturale. "Implicazioni Sociali" sottolinea la necessità di comprendere le conseguenze concrete di questa tecnologia sulla struttura delle nostre società.

---

**Note Aggiuntive per il Creativo dell’Organizzazione:**

*   **Pubblico Target:** Questi titoli sono pensati per un pubblico che include antropologi, sociologi, esperti di tecnologia, studenti e chiunque sia interessato a esplorare le implicazioni sociali della GenAI.
*   **Tono:** Ho cercato di bilanciare la serietà con un pizzico di curiosità e stimolo al pensiero critico.
*   **Elemento Chiave:** Tutti i titoli sottolineano l'importanza dell'approccio antropologico per comprendere questi cambiamenti, evidenziando che non si tratta solo di una questione tecnica, ma di come queste tecnologie stanno influenzando la nostra umanità.

Spero che queste proposte siano un buon punto di partenza! Fammi sapere se hai bisogno di ulteriori variazioni o idee.

---
**Analista**: Ottima domanda! Dopo aver valutato attentamente le opzioni, ritengo che i due titoli migliori siano:

**1. "L'Uomo Rivisto: Antropologia e l’Era della Generative Intelligence"**

**2. “Oltre l’Algoritmo: Antropologia, Intelligenza Artificiale Generativa e il Futuro dell’Identità”**

Ecco la mia giustificazione dettagliata per questa scelta:

**1. "L'Uomo Rivisto: Antropologia e l’Era della Generative Intelligence"**

*   **Impatto e Risonanza:** Questo titolo ha una forza immediata e un'impronta significativa. L'uso di “Rivisto” implica un cambiamento profondo, una trasformazione fondamentale del nostro modo di pensare all’essere umano, il che cattura l'attenzione e genera curiosità.
*   **Chiarezza dell’Obiettivo:** La combinazione con "Antropologia" è cruciale perché definisce immediatamente lo scopo del seminario. Non si tratta solo di tecnologia, ma di un’analisi da parte della disciplina più appropriata per studiare l'impatto umano.
*   **Rilevanza per il Futuro:** “Era della Generative Intelligence” colloca il tema in un contesto contemporaneo e futuro, sottolineando la portata dell'innovazione tecnologica che stiamo affrontando.

**2. “Oltre l’Algoritmo: Antropologia, Intelligenza Artificiale Generativa e il Futuro dell’Identità”**

*   **Focus sulla Questione Centrale:** Questo titolo affronta direttamente una delle preoccupazioni più pressanti legate alla GenAI: come questa tecnologia sta rimodellando la nostra identità. L'identità è un concetto centrale nell'antropologia, quindi l'associazione con la disciplina è estremamente pertinente.
*   **Invito al Pensiero Critico:** "Oltre l’Algoritmo" suggerisce che ci sia qualcosa di più profondo da esplorare rispetto alle mere analisi tecniche dell’IA. Questo incoraggia i partecipanti a riflettere sulle implicazioni culturali e sociali della tecnologia.
*   **Connessione con il Futuro:** La frase finale – "Futuro dell'Identità" – stabilisce un orizzonte temporale chiaro e sottolinea l'urgenza di comprendere queste trasformazioni prima che siano troppo tardi.

**Perché ho scelto questi due invece degli altri?**

Mentre tutti i titoli sono validi, questi due offrono il miglior equilibrio tra:

*   Chiarezza del focus antropologico
*   Rilevanza per le questioni chiave sollevate dalla GenAI
*   Potenziale per stimolare un dibattito critico e riflessivo

Spero che questa giustificazione sia soddisfacente! Fammi sapere se hai altre domande o preferenze.

---
**Decisore**: Dopo aver rivalutato tutte le opzioni, ritengo che il titolo migliore sia:

**“L'Uomo Rivisto: Antropologia e l’Era della Generative Intelligence”**

Ecco la mia giustificazione dettagliata per questa scelta finale:

*   **Forza del Messaggio:** Questo titolo ha una forza comunicativa immediata e potente. L'uso di “Rivisto” evoca un cambiamento fondamentale, un’alterazione profonda nella nostra comprensione dell’essere umano. Non è un semplice aggiornamento, ma una vera e propria revisione delle nostre categorie concettuali.
*   **Chiarezza e Definizione:** La combinazione con "Antropologia" è assolutamente cruciale. Definisce immediatamente lo scopo del seminario: non si tratta solo di analizzare l'IA generativa come tecnologia, ma di studiarne le implicazioni per la natura umana attraverso la lente dell’antropologia.
*   **Rilevanza e Prospettiva:** “Era della Generative Intelligence” colloca il tema in un contesto contemporaneo e futuro, sottolineando la portata dell'innovazione tecnologica che stiamo affrontando. Evita di concentrarsi esclusivamente sulle potenzialità positive dell’IA, riconoscendo implicitamente le sfide e i rischi.
*   **Appeal Generale:** Il titolo è abbastanza ampio da attirare un pubblico diversificato – antropologi, sociologi, esperti di tecnologia, studenti – senza essere eccessivamente specifico per limitare l'interesse.

**Perché ho scartato gli altri titoli?**

Sebbene “Oltre l’Algoritmo…” sia un titolo stimolante e pertinente, il suo focus sulla "identità" potrebbe risultare troppo ristretto per un seminario che affronta una gamma così ampia di implicazioni antropologiche.  “La Macchina come Mediatore” era un po’ più tecnico e meno evocativo.

**In sintesi:** “L'Uomo Rivisto…” è il titolo migliore perché offre la combinazione perfetta di forza, chiarezza, rilevanza e appeal per il pubblico target che cerchiamo di raggiungere. È un titolo che invita all’esplorazione, alla riflessione critica e alla comprensione profonda delle implicazioni dell'IA generativa sul futuro dell'umanità.

Spero che questa decisione finale sia in linea con le tue aspettative!